In [3]:
import scVI
import tensorflow as tf
from benchmarking import *
from helper import *

import numpy as np
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pickle

## Load Data

In [2]:

X_type = pd.read_csv("../../data/cell.type_E31.csv", sep=",")
cell_types, cluster_labels = np.unique(np.array(X_type)[:,1], return_inverse=True)

print("Number of cluster labels",len(cluster_labels))
    

Number of cluster labels 3381


In [13]:

# dge = np.load("../../data/dge_E31.dat")
with open("/home/zgy_ucla_cs/Research/singleCell/scRNA-Seq-TCC-prep/mat_21mer/TCC_matrix.dat", 'rb') as f:
    T=pickle.load(f, encoding='latin1')
    
# with open("/home/zgy_ucla_cs/Research/singleCell/scRNA-Seq-TCC-prep/mat_31mer/TCC_matrix.dat", 'rb') as f:
    T=pickle.load(f, encoding='latin1')    
# Get the expression matrix: Cell * Genes

# X = dge.values.T
# cell_names = dge.columns.values
# cell_names2 = np.array(X_type)[:,0]

In [14]:
T.shape

(279478, 3381)

In [23]:
X = T.T.toarray()

In [27]:
X.shape

(3381, 279478)

In [29]:

expression_train, expression_test, c_train, c_test = train_test_split(X, cluster_labels, random_state=0)

In [31]:
expression_train.shape

(2535, 279478)

In [17]:
log_library_size = np.log(np.sum(expression_train, axis=1))
mean, var = np.mean(log_library_size), np.var(log_library_size)

batch_size = 128
learning_rate = 0.001
epsilon = 0.01
latent_dimension = 10


In [18]:
tf.reset_default_graph()
expression = tf.placeholder(tf.float32, (None, expression_train.shape[1]), name='x')
kl_scalar = tf.placeholder(tf.float32, (), name='kl_scalar')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=epsilon)
training_phase = tf.placeholder(tf.bool, (), name='training_phase')

# getting priors
log_library_size = np.log(np.sum(expression_train, axis=1))
mean, var = np.mean(log_library_size), np.var(log_library_size)
# loading data
model = scVI.scVIModel(expression=expression, kl_scale=kl_scalar, \
                         optimize_algo=optimizer, phase=training_phase, \
                          library_size_mean=mean, library_size_var=var, n_latent=latent_dimension)

#starting computing session
sess = tf.Session()

Running scVI on 279478 genes
scVI will run without batch correction
Will work on mode numeric for incorporating library size
Will work on mode gene for modeling inverse dispersion param
Will apply zero inflation
1 hidden layers at 128 each for a final 10 latent space


In [ ]:
sess.run(tf.global_variables_initializer())
result = train_model(model, (expression_train, expression_test), sess, 250, batch_size=batch_size)